In [1]:
import pandas as pd
import numpy as np
import dash
import plotly
import pickle
import math
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

In [2]:
sdg_text=['No Poverty',
        'Zero Hunger', 
        'Good Health and Well-being',
        'Quality Education', 
        'Gender Equality', 
        'Clean Water and Sanitation', 
        'Affordable and Clean Energy', 
        'Decent Work and Economic Growth', 
        'Industry, Innovation and Infrastructure', 
        'Reduced Inequality', 
        'Sustainable Cities and Communities', 
        'Responsible Consumption and Production', 
        'Climate Action', 
        'Life Below Water', 
        'Life on Land', 
        'Peace and Justice Strong Institutions', 
        'Partnerships to achieve the Goal' ]

sdg_color=['rgb(229, 36, 59)',
            'rgb(221, 166, 58)',
            'rgb(76, 159, 56)',
            'rgb(197, 25, 45)',
            'rgb(255, 58, 33)',
            'rgb(38, 189, 226)',
            'rgb(252, 195, 11)',
            'rgb(162, 25, 66)',
            'rgb(253, 105, 37)',
            'rgb(221, 19, 103)',
            'rgb(253, 157, 36)',
            'rgb(191, 139, 46)',
            'rgb(63, 126, 68)',
            'rgb(10, 151, 217)',
            'rgb(86, 192, 43)',
            'rgb(0, 104, 157)',
            'rgb(25, 72, 106)' ]

In [4]:
data = pd.read_pickle('kfw-prototyp/Data/data.pkl')
data = data.fillna(0)
sums = data.iloc[: ,1:188].apply(sum, axis = 0)
sums_df = pd. DataFrame(sums).T

sdgs = []

for col in sums_df.columns:
    if 'SDG' in str(col):
        sdgs.append(col)

sdg = sums_df[sdgs]

sdg_sums = (sdg.T).rename(columns={0: 'sum'})

In [5]:
def mapping(x):
    y = []
    for i in list(x):
        if i == x.max():
            i=100
        else:
            i=round(i/x.max()*100,2)
        y.append(i)
    return y

In [7]:
sdg_sums['mapping'] = mapping(sdg_sums['sum'])
values = sdg_sums['mapping']
df = sdg_sums

In [8]:
# Get incremental angle value
n = len(df) + 20
dtheta = 2*math.pi / n
theta = math.pi / 2

# Initialise
xcoord = []
ycoord = []

In [9]:
# This is for the white - circle in the middle
adjust =  20
traces = []


for i in range(len(df)):
    
    a = df.mapping[i] + adjust
    x1 = adjust * math.cos(theta)
    y1 = adjust * math.sin(theta)
  
    x2 = a * math.cos(theta)
    y2 = a * math.sin(theta)
  
    xcoord.append(x1)
    xcoord.append(x2)
    xcoord.append(np.nan)
    ycoord.append(y1)
    ycoord.append(y2)
    ycoord.append(np.nan)
    #cols = [cols, df$Continent[ctr], df$Continent[ctr], NA)
  
    theta = theta + dtheta
  
    traces.append(go.Scatter(x = [x1, x2],
                 y = [y1, y2],
                 mode = "lines", 
                 name = sdg_text[i],
                 line = dict(width = 10, color = sdg_color[i])
                                ))
    

up = np.array([x for x in xcoord if str(x) != 'nan']+[x for x in ycoord if str(x) != 'nan']).max()+10
down = np.array([x for x in xcoord if str(x) != 'nan']+[x for x in ycoord if str(x) != 'nan']).min()-10

In [10]:
app = dash.Dash()

app.layout = html.Div([
    html.H1('Aufteilung der Kredite in SDGs'),
    dcc.Graph(id = 'figure1',
            figure= {'data': traces,
                     'layout': go.Layout(title='KfW SDG contribution - new commitment volume year 2018',
                                xaxis={'range': [down, up], 'domain': [0, 1], 'title':"", 'showgrid': False, 'zeroline':False, 'showticklabels': False},
                                yaxis={'range': [down, up], 'domain': [0, 1], 'title':"", 'showgrid': False, 'zeroline':False, 'showticklabels': False},                                  showlegend=False, width=800, height=800, 
                                template=None),
                     'shape': dict( type="circle",
                                    x0= ( - adjust),
                                    y0= ( - adjust),
                                    x1= ( adjust),
                                    y1= ( adjust),
                                    fillcolor= "white",
                                    line=dict(color="black", width=2),
                                    xref='x', yref='y')          
                    } 
            )
])

In [12]:
if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2020 12:43:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 12:43:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 12:43:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 12:43:04] "GET /_favicon.ico?v=1.9.1 HTTP/1.1" 200 -
